In [13]:
# COMSOL mesh loading to Python
# Matlab .m codes in IRINA/Connect_with_COMSOL/COMSOL_Python directory are used for mesh generation
# All geometries are saved to IRINA/Meshes

from Connect_with_COMSOL.COMSOL_Python.mesh_loading import load_meshes
load_meshes()

In [ ]:
# VAE training using all meshes (latent vector length is to be stated by the user)
# All .csv geometries are used from IRINA/Meshes directory
# Using all meshes VAE outputs the whole space to be explored (in the form of gene variances)
# Using target meshes VAE outputs the space to pay attention to (as well as gene importance influencing mutations and crossovers)

latent_vector_length = 16

from Models.VAE_model.run_vae_model import get_vae_latent_vectors
latent_vectors_target = get_vae_latent_vectors(latent_vector_length=latent_vector_length)

In [ ]:
# All meshes are used to calculate the whole space to be explored (it will limit the mutations)
# Target meshes are used to calculate the space to pay attention to (it will be used for individual initialization)

from Models.rvGA_model.rvga_functions import gene_boundaries_calculation

gene_boundaries_all = gene_boundaries_calculation(latent_vectors_df=latent_vectors_all,
                                                  latent_vector_length=latent_vector_length)
gene_boundaries_target = gene_boundaries_calculation(latent_vectors_df=latent_vectors_target,
                                                     latent_vector_length=latent_vector_length)

In [ ]:
# Gene importance is calculated for the target meshes (it will be used to tune mutation and crossover rate)
# max_diff_in_mating parameter allows to tune the influence of gene value variance on mutations and crossovers (it is to be set by the user)

max_diff_in_mating = 2

from Models.rvGA_model.rvga_functions import gene_importance_correction

gene_importance_target = gene_importance_correction(latent_vectors_df=latent_vectors_target,
                                                    max_diff_in_mating=max_diff_in_mating)

In [ ]:
# Genetic algorithm is run:
#
# population_size (number of individuals in each generation),
# max_generations (maximal number of generations after which the algorithm stops),
# spectrum_proximity_percentage (max spectrum proximity to be achieved by the algorithm),
# percentage_of_survived (percentage of individuals surviving the tournament),
# prob_mutation (default probability of mutation for gene with importance equal to 1),
# and prob_crossover (default probability of crossover for gene with importance equal to 1) are to be set by the user
#
# Outputs the final generation

from Models.rvGA_model.run_rvga_model import run_genetic_algorithm

x_evolved = run_genetic_algorithm(latent_vectors_target=latent_vectors_target,
                                  vae_latent_vector_length=latent_vector_length,
                                  gene_boundaries=gene_boundaries_target,
                                  gene_importance=gene_importance_target,
                                  population_size=50,
                                  max_generations=50,
                                  spectrum_proximity_percentage=90,
                                  percentage_of_survived=50,
                                  prob_mutation=10,
                                  prob_crossover=80)